# Transformer 

Transformer 是一种基于 ​​自注意力机制（Self-Attention）​​ 的深度学习模型，由 Google 在 2017 年的论文《Attention Is All You Need》中提出。它彻底改变了自然语言处理（NLP）领域，并逐步扩展到计算机视觉、语音识别等多个领域。

------

## 原理介绍

### 什么是注意力机制

注意力机制（​​Attention Mechanism​​）是深度学习中一种模拟人类视觉或认知聚焦行为的技术，它允许模型在处理数据时动态地​​关注与当前任务最相关的部分​​，而忽略无关信息。其核心思想是​​通过权重分配，强调重要特征，弱化次要特征​​。

#### 核心数学形式​​

注意力机制通常分为三步（以经典的 ​​Scaled Dot-Product Attention​​ 为例）：

+ ​​计算注意力分数​​：
对查询（Query）和键（Key）做相似度计算，得到权重。
$$
Attention Score= \frac {Q⋅K^T} {\sqrt d_k}
$$
（d_k 是 Key 的维度，缩放用于防止梯度消失）
+ 归一化为权重（Softmax）​​：
$$Attention Weights=softmax(Attention Score)$$
+ ​​加权求和值（Value）​​：
$$Output=Attention Weights⋅V$$

#### 举例解释

+ 假设一句话为 "the cat sat on the table", 每一个单词都可以用一个 $1 \times N$ 的嵌入向量表示，这样这句话就表示成了 $X = 6 \times N$ 的矩阵
+ 训练三个权重向量 ($N \times 1$) 分别为 $W_Q, W_K, W_V$, 计算出查询，键和值 $Q = X * W_Q, K = X * W_K, V = X * W_V$
+ 计算每一个单词的注意力得分，并和 V 相乘，得到最终的输出

--------


### 注意力机制的类型

1. 自注意力 (self-attention)
    Query, Key, Value 来自同一输入，用来捕捉序列内部的关系（比如句子中单词的远距离依赖）
    ```python
    self_atten = nn.MultiheadAttention(embed_dim=64, num_head=4)
    output, _ = self_atten(query, key, value)
    ```
2. 交叉注意力 (cross-attention)
    Query 来自当前序列，Key， Value来自预训练的序列 （如机器翻译中源语言到目标语言）
3. 多头注意力 (multi-head attention)
    将 Q/K/V 分成不同的头，并行计算后拼接，实现对不同子空间的特征捕捉能力

--------


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np


# 超参数
batch_size = 64
patch_size = 7      # 将图像分割成 7x7 的patch 小块
embed_size = 64     # 嵌入向量的维度
num_heads = 4       # 注意力头数
num_classes = 10
epochs = 5

# 数据加载
transform = transforms.Compose([
    transforms.ToTensor(),                  # 步骤1: 将图像转为 Tensor 格式
    transforms.Normalize((0.5,), (0.5,))    # 步骤2: 归一化处理，使数据符合均值为0.5，标准差也是0.5
])
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, bitch_size=batch_size, shuffle=True)

# 定义 Transformer 模型
class VisionTransformer(nn.Module):
    def __init__(self):
        super().__init__()
        self.patch_embed = nn.Conv2d(1, embed_size, kernel_size=patch_size, stride=patch_size)
        num_patches = (28 // patch_size) ** 2
        self.pos_embed = nn.Parameter(torch.randn(1, num_patches + 1, embed_size))
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_size))
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=embed_size, nhead=num_heads),
            num_layers=2
        )
        self.fc = nn.Linear(embed_size, num_classes)

    def forward(self, x):
        x = self.patch_embed(x).flatten(2).transpose(1, 2)

        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embed

        x = self.transformer(x)

        cls_output = x[:, 0]
        return self.fc(cls_output)

在上面的模型中，主要分成了几个步骤

**1. 图像分块**

+ 将 28x28 的图像分割为 ​​16 个 7x7 的 patch​​，每个 patch 通过卷积（nn.Conv2d）映射为 embed_dim 维向量。
+ 输入​​：(B, 1, 28, 28) → ​​输出​​：(B, 16, embed_dim)。

**2. 位置Token和分类Token**

+ ​分类 Token (cls_token)​​：添加到 patch 序列开头,用于最终分类。在 transformer 中主要是替代池化层获取所有patch的信息，cls_token 与任何patch没有关系，通过注意力权重学习哪些patch对分类更重要
+ ​​位置编码 (pos_embed)​​：告诉模型每个 patch 的原始位置（因为 Transformer 本身无序）。Transformer 是对patch的输入顺序不敏感的，也就是说无论怎么判列，反正最后通过注意力会选取哪些patch对分类更重要，所以需要位置编码来告诉模型每个patch的原始位置。标注各个patch之间的关系，比如猫喜欢吃鱼，但不能说鱼喜欢吃猫，不添加顺序可能有歧义

**3. Transformer编码器**

+ 通过多层 TransformerEncoder 处理序列，捕捉 patch 间的关系。
+ ​​输出​​：(B, 17, embed_dim)（16 patches + 1 cls_token）。

4. 分类头

+ 取 cls_token 对应的输出向量，通过全连接层 (nn.Linear) 分类。

In [ ]:
model = VisionTransformer()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 训练模型
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item():.4f}')

# 测试（简化版）
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total:.2f}%')